In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*aw+198*aj+222*ap+(4.44/18*1000)*bw+198*bj+222*bp
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+aj+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints
con8 = m.addConstr(aw>=0)
con9 = m.addConstr(aj>=0)
con10 = m.addConstr(ap>=0)
con11 = m.addConstr(bw>=0)
con12 = m.addConstr(bj>=0)
con13 = m.addConstr(bp>=0)

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [14400, 1000, 2000, 600, 2400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Academic license - for non-commercial use only - expires 2022-07-19
Using license file C:\Users\jkettune\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 6 columns and 22 nonzeros
Model fingerprint: 0x1ec5877e
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 11 rows and 3 columns
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4000000e+05   9.624497e+02   0.000000e+00      0s
       2    6.7606667e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective  6.760666667e+05

Profit contribution: 676067

Decision variables:
aw = 525
aj = 75
ap = 0
bw = 175
bj = 225
bp = 2000


  Name  Final Value  Reduced Cos